# VAE
Variational autoencoder [1] models inherit autoencoder architecture, but make strong assumptions concerning the distribution of latent variables. They use variational approach for latent representation learning

[1] Auto-Encoding Variational Bayes, Diederik P Kingma, Max Welling 2013

In [ ]:
# Setup
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.slim import fully_connected as fc

In [ ]:
# Load MNIST data
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
num_data = mnist.train.num_examples

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [ ]:
# Hyperparamters. Your job to find these.
num_epochs = 
batch_size = 
z_dim = 
lr = 

# Encoder 
Implement the encoder network $Q_\phi(z \mid x)$. Use Tensorflow's `fully_connected` function [API guide](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/fully_connected) to write less lines or code. Use 3 shared FC layers and 1 FC layer at the end for the mean $\mu(X_i)$ and another 1 FC layer at the end for the diagonal variance $\sigma(X_i)$.

Also, implement the reparameterization trick we talked about in the class.

- Do take a look at the API and make sure to not use any non-linearities at the final layer!
- Make it return log of the variance so we work with smaller numbers (for stability)!

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])

def Q(X):
    # Your code here for encoder network.
    return z_mu, z_logvar
    
def sample_z(mu, log_var):
    # Your code here for the reparameterization trick.
    # reparameterized_sample = ...
    return reparameterized_sample

# Decoder
Implement the decoder network $P_\theta(X \mid z)$. Use Tensorflow's `fully_connected` function [API guide](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/fully_connected) to write less lines or code as well.

In this exercise, we will use Bernoulli MLP decoder explained in Appendix C.1 in the original paper.

Use 3 FC layers and 1 FC layer at the end for the unnormalized and normalized logits.

In [ ]:
def p(z):
    # Your code here for the decoder network.
    return prob, logits

# Rest of the graph
We can define the rest of the network as below.

In [ ]:
z_mu, z_logvar = Q(X)
z_sample = sample_z(z_mu, z_logvar)
X_samples, logits = P(z_sample)

# Graph loss function
You might find `tf.nn.sigmoid_cross_entropy_with_logits` function [API guide](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits) useful.

In [ ]:
# Your code here for the reconstruction loss
recon_loss = 
# Your code here for the closed form KL loss discussed in class
kl_loss = 

# Total loss
vae_loss = recon_loss + kl_loss

In [ ]:
# Optimizer and solver
solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(vae_loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Train a _great_ generative model on MNIST!

Now, it's your job to experiment with hyperparameters defined in the beggining. The code below will also generate images and save the results to `figures/` directory. We are attaching the example images from our run and we expect your images to be of similar quality. 
![](out/080.png)


In [ ]:
if not os.path.exists('figures/'):
    os.makedirs('figures/')

for epoch in range(num_epochs):
    for it in range(num_data // batch_size):
        X_mb, _ = mnist.train.next_batch(batch_size)

        _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb})

    if epoch % 5 == 0:
        print('Epoch: {}, Loss: {:.4}'. format(epoch, loss))

        samples = sess.run(X_samples, feed_dict={z_sample: np.random.randn(16, z_dim)})

        fig = plot(samples)
        plt.savefig('figures/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)